In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [3]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [4]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [6]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x000001546F116B80>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x000001546F116B80>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x000001546F116B80>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [7]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [04:12<00:00,  1.26s/it]
[MainProcess/INFO] experiments finished


In [8]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,78,127.776961,1.0,0.177478,105.079776,1.0,0.001063,100.351911,1.5,0.382137,...,5,0,2,3,6,4,2,4,0,dikesnet
1,6,52.187777,1.5,0.728822,315.451488,1.5,0.617916,290.066362,1.5,0.411734,...,5,0,2,3,6,4,2,5,0,dikesnet
2,11,222.333560,1.0,0.527457,107.358534,10,0.969164,200.876688,10,0.040427,...,5,0,2,3,6,4,2,6,0,dikesnet
3,5,318.948618,10,0.892231,30.179748,1.5,0.106845,174.608902,1.0,0.815942,...,5,0,2,3,6,4,2,7,0,dikesnet
4,63,290.656968,1.0,0.066433,297.082066,1.0,0.848614,158.887259,10,0.673972,...,5,0,2,3,6,4,2,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,114,170.105091,1.5,0.055530,155.986492,1.0,0.649141,114.199459,1.5,0.156400,...,1,4,1,7,3,1,0,49,3,dikesnet
196,71,213.652723,1.5,0.257191,279.796650,1.0,0.085365,148.789599,10,0.544143,...,1,4,1,7,3,1,0,50,3,dikesnet
197,55,115.051924,10,0.636935,115.881185,1.0,0.785788,195.545312,1.5,0.228004,...,1,4,1,7,3,1,0,51,3,dikesnet
198,122,248.121418,1.0,0.657179,162.016448,1.5,0.057754,341.263938,1.0,0.494835,...,1,4,1,7,3,1,0,52,3,dikesnet


In [16]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{' 0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 2', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':5}),
           Policy('policy 3', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':10}),
            Policy('policy 4', **{'0_RfR 0':1,
                                 '0_RfR 1': 0,
                                '0_RfR 2': 0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 5', **{'0_RfR 0':0,
                                 '0_RfR 0':1,
                                '0_RfR 0':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 6', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
        Policy('policy 7', **{' 1_RfR 0':1,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 8', **{'1_RfR 0':0,
                                 '1_RfR 1':1,
                                '1_RfR 2':0,
                                'A.5_DikeIncrease 0':0}),
           Policy('policy 9', **{'1_RfR 0':0,
                                 '1_RfR 1':0,
                                '1_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
            Policy('policy 10', **{'2_RfR 0':1,
                                 '2_RfR 1':0,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 11', **{'2_RfR 0':0,
                                 '2_RfR 1':1,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 12', **{'2_RfR 0':0,
                                 '2_RfR 1':0,
                                '2_RfR 2':1,
                                'A.3_DikeIncrease 0':0})
            ]

In [17]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

results



[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 12 policies * 1 model(s) = 1200 experiments
100%|██████████████████████████████████████| 1200/1200 [10:02<00:00,  1.99it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


(      A.0_ID flood wave shape    A.1_Bmax A.1_Brate  A.1_pfail    A.2_Bmax  \
 0                          72  107.241711       1.0   0.913511  167.102659   
 1                          64  169.770559       1.5   0.941424   86.678339   
 2                          21  173.002353       1.5   0.147531  233.826818   
 3                          95  146.798872        10   0.973705   56.452639   
 4                         122  144.858401        10   0.152137  310.315219   
 ...                       ...         ...       ...        ...         ...   
 1195                       12  195.861492       1.5   0.204755  287.766755   
 1196                      102  116.516462        10   0.223526  260.494436   
 1197                       44  115.470371       1.0   0.765844  294.084128   
 1198                      117  287.959349        10   0.488778  113.016431   
 1199                      132  165.214851        10   0.340293  346.613834   
 
      A.2_Brate  A.2_pfail    A.3_Bmax A.3_Brate  

In [18]:
import pandas as pd
a,b = results
df = pd.DataFrame(b)
df

#a = np.min(df['A.3 Total Costs'])#



,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,9.506030e+07,0.000000,1.118850e+08,0.002483,1.389003e+08,0.231006,2.485405e+07,0.004771,2.983051e+08,0.220732,0.0,0.0
1,9.506030e+07,0.000000,7.323512e+08,0.478150,1.959433e+07,0.000000,1.686899e+07,0.000000,7.344192e+07,0.001258,0.0,0.0
2,2.512252e+09,1.652505,1.093152e+08,0.000000,9.549722e+07,0.131235,1.686899e+07,0.000000,7.197799e+07,0.000000,0.0,0.0
3,9.506030e+07,0.000000,1.260903e+08,0.017792,6.827920e+07,0.097941,4.256294e+07,0.013238,1.166882e+08,0.046773,0.0,0.0
4,2.508217e+09,1.649032,1.093152e+08,0.000000,2.421301e+08,0.400379,4.272025e+07,0.011854,7.197799e+07,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2.679112e+09,1.903257,1.022563e+08,0.000000,5.437894e+07,0.065691,1.623000e+07,0.000000,4.542000e+07,0.000000,30700000.0,0.0
1196,2.508319e+09,1.885683,1.022563e+08,0.000000,1.858000e+07,0.000000,7.595766e+07,0.024761,4.542000e+07,0.000000,30700000.0,0.0
1197,8.964065e+07,0.002453,1.070046e+08,0.004018,3.478393e+07,0.025415,4.141514e+07,0.011307,4.542000e+07,0.000000,30700000.0,0.0
1198,3.159608e+08,0.177236,2.741476e+08,0.169799,1.858000e+07,0.000000,8.926443e+07,0.036158,4.542000e+07,0.000000,30700000.0,0.0


In [14]:
print(np.min(df['A.3 Total Costs']))
print(np.average(df['A.3 Total Costs']))
print(np.max(df['A.3 Total Costs']))
print(np.percentile(df['A.3 Total Costs'], 95))

18580000.00000099
465006056.3632478
2586846873.7498918
2030579964.2472858


In [103]:
print(np.min(df['A.3_Expected Number of Deaths']))
print(np.average(df['A.3_Expected Number of Deaths']))
print(np.max(df['A.3_Expected Number of Deaths']))

0.0
0.08017002562217167
0.4092784227625138


In [104]:
print(np.min(df['RfR Total Costs']))
print(np.average(df['RfR Total Costs']))
print(np.max(df['RfR Total Costs']))

256100000.00000003
256100000.00000003
256100000.00000003


In [105]:
print(np.min(df['Expected Evacuation Costs']))
print(np.average(df['Expected Evacuation Costs']))
print(np.max(df['Expected Evacuation Costs']))

3142.0938808781575
44874.87715596213
133733.3544270394
